In [2]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

In [3]:
def database_gen():
    #puntos
    nx=100
    ny=100
    nt=100
    #dimensiones
    Lx=1.0
    Ly=1.0
    #tiempo total
    T=1.0
    #Generacion de datos
    A=1.0
    kx=2*np.pi
    ky=2*np.pi
    f_t=1.0
    phi=0.0
    sigma=0.05
    seed=2
    
    channel_first=True
    dtype=np.float32
    
    rng = np.random.default_rng(seed)
    x = np.linspace(0, Lx, nx, dtype=dtype)
    y = np.linspace(0, Ly, ny, dtype=dtype)
    t = np.linspace(0, T, nt, dtype=dtype)
    X, Y = np.meshgrid(x, y, indexing='xy')  # (ny, nx)

    u = np.empty((nt, ny, nx), dtype=dtype)
    v = np.empty((nt, ny, nx), dtype=dtype)
    
    for i, ti in enumerate(t): #bucle que recorre cada instante t 
        phase_t = 2*np.pi*f_t*ti
        u_clean = A * np.sin(phase_t) * np.cos(kx*X) * np.cos(ky*Y)
        v_clean = A * np.cos(phase_t + phi) * np.sin(kx*X) * np.sin(ky*Y)
        u[i] = u_clean + rng.normal(0.0, sigma, size=(ny, nx))
        v[i] = v_clean + rng.normal(0.0, sigma, size=(ny, nx))
    
    F = np.stack([u, v], axis=1)
    
    np.savez_compressed("campo_vel.npz", F=F)
    return F

F = database_gen()
F.shape

(100, 2, 100, 100)

La manera en la que se genera el array de la database el F(tiempo, canal, pos. y, pos. x).
Canales: 0 > u
         1 > v

In [4]:
# Definimos la clase para transformar los datos a dataset

class uv_data_2_dataset(Dataset): #vamos a devolver toda la informacion disponible en cada instante de tiempo: para cada t --> tensor (2, ny, nx)
    def __init__(self, path_to_data, mmap=True, dtype=torch.float32):
        self.raw_data = np.load(path_to_data, mmap_mode='r' if mmap else None) #la opción "r" evita cargar todo en RAM
        self.F = self.raw_data["F"]
        assert self.F.ndim == 4 and self.F.shape[1] == 2 #comprobacion de dimensiones correctas
        self.dtype = dtype
        
    def __len__(self): #tiene que devolver el numero de muestras disponobles para darle al DataLoader
        return self.F.shape[0]
    
    def __getitem__(self, index): #esta es la funcion que le da los items al DataLoader. 
        f_np = self.F[index]
        f = torch.from_numpy(f_np).to(self.dtype) #lo pasamos de numpy a torch y al type deseado
        return f
     
        

In [9]:
# Cargamos los datos

dataset_campo_velocidades = uv_data_2_dataset("campo_vel.npz")

data_load_campo_velocidades = DataLoader(
    dataset_campo_velocidades, 
    batch_size=5,
    shuffle=True,
    num_workers=4
)

In [6]:
batch = next(iter(data_load_campo_velocidades))
batch.shape

torch.Size([10, 2, 100, 100])

In [12]:
for batch_id, F in enumerate(data_load_campo_velocidades):
    print(batch_id)
    print(F.shape)

0
torch.Size([5, 2, 100, 100])
1
torch.Size([5, 2, 100, 100])
2
torch.Size([5, 2, 100, 100])
3
torch.Size([5, 2, 100, 100])
4
torch.Size([5, 2, 100, 100])
5
torch.Size([5, 2, 100, 100])
6
torch.Size([5, 2, 100, 100])
7
torch.Size([5, 2, 100, 100])
8
torch.Size([5, 2, 100, 100])
9
torch.Size([5, 2, 100, 100])
10
torch.Size([5, 2, 100, 100])
11
torch.Size([5, 2, 100, 100])
12
torch.Size([5, 2, 100, 100])
13
torch.Size([5, 2, 100, 100])
14
torch.Size([5, 2, 100, 100])
15
torch.Size([5, 2, 100, 100])
16
torch.Size([5, 2, 100, 100])
17
torch.Size([5, 2, 100, 100])
18
torch.Size([5, 2, 100, 100])
19
torch.Size([5, 2, 100, 100])
